In [1]:
import json
import openai
import os
import re
from langchain_openai import ChatOpenAI
import transformers
from datasets import Dataset

In [261]:
file_path = 'C:\\Users\\PC\\Desktop\\DoYoung\\DS\\github\\bitamin_auto_readme_generator\\data\\object_detection\\output\\ocr_samples_txt\\trading_text.txt'

with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

In [262]:
api_key_filepath = "C:\\Users\\PC\\Desktop\\DoYoung\\DS\\비타민NLP_240701\\text_summarization\\openai_api_key.json"
with open(api_key_filepath, 'r') as f:
    api_key = json.load(f)
api_key = api_key['OPENAI_API_KEY']

In [263]:
os.environ['OPENAI_API_KEY'] = api_key

In [264]:
llm_3 = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=os.environ['OPENAI_API_KEY']
)

In [265]:
llm_4 = ChatOpenAI(
    model="gpt-4o",
    openai_api_key=os.environ['OPENAI_API_KEY']
)

### STEP1: 첫 5페이지의 텍스트만 추출

In [266]:
end_marker = text.find('<p.6>')

if end_marker != -1:
    text5 = text[:end_marker]
else:
    text5 = text

### STEP2: 마크다운 형식으로 팀원, 주제, main topic 추출

In [267]:
def extract_info(text5):
    prompt = f"""
    Extract the following information from the provided text:
    1. Name (excluding the team name, include all people listed)
    2. Title (include the entire title as it appears in the text)
    3. Main Topics (only main topics from the Table of Contents, excluding subtopics)

    The Main Topics should be extracted from the section that follows headers such as 'TABLE OF CONTENTS', '목차 소개', or any similar variation.
    Ensure that team member's name are not split into multiple lines. Name should be connected by commas if there are more than one.
    Ensure that title is not cut off and is extracted in their entirety.
    Exclude any subtopics or secondary information while extracting main topics.
    Stop at the next page marker '<p.'.

    Do not include any additional notes or explanations in the output.

    Text: {text5}


    Format the extracted information as follows:
    <subject>title</subject>
    <team>team members</team>
    <index>main topics</index>

    """

    response = llm_4.invoke(prompt)
    extracted_info = response.content.strip()
    return extracted_info

extracted_info = extract_info(text5)
print(extracted_info)


<subject>을홀 '시' 계을 |측 과 학 할용 한 피미 여 강호 습을 시 템 트 레 이딩 익 수 극도 화 BTAMIN PROJECT</subject>
<team>송규헌, 송휘종, 서영우, 이태경, 정유진, 정준우</team>
<index>주제, NorkFlow, 종목 선정, 시스템 트레이딩, 데이터수집및 전처리, 시계열 모델, 강화학습 모델, 모델 평가및 선정, 시스템 트레이딩, 결과</index>


In [268]:
main_topics_match = re.search(r'<index>(.*?)</index>', extracted_info, re.DOTALL)
if main_topics_match:
    main_topics = main_topics_match.group(1).strip()
else:
    main_topics = ""

### STEP3: 전체 텍스트에서 대주제, 소주제, 세부내용 추출

In [269]:
def extract_detrails(text, main_topics):
    prompt = f"""
    Extract detailed information for each main topic and its subtopics from the provided text.
    Do not change the language of the original input text. 
    Focus on the key details and results relevant to the project, with particular emphasis on the final model outcomes. 
    Ensure that the final model results are described with a high level of accuracy and detail, including specific metrics, performance evaluations, and any significant findings.
    Include the **first page** where each subtopic begins in the "pages used" list. For example, if a subtopic spans from page 7 to page 10, only include page 7 in the result.
    Ensure that detailed contents are extracted comprehensively from all relevant pages, leaving no key information behind. 
    Focus on capturing detailed content across all pages, ensuring that nothing is missed, especially in relation to the provided main topics.

    Format the extracted information as follows:
    <main>main topic</main>
    <sub>subtopic</sub> <content>detailed contents</content> <page>3</page>
    <sub>subtopic</sub> <content>detailed contents</content> <page>5</page>

    <main>main topic</main>
    <sub>subtopic</sub> <content>detailed contents</content> <page>10</page>
    <sub>subtopic</sub> <content>detailed contents</content> <page>11</page>

    Use the main topics provided below:
    {main_topics}
    
    The text is divided into pages using the format <p.number>. For example, page 2 is marked as <p.2>. 
    Make sure to extract text from all pages except for the first five pages to ensure no information is missed.
    If any part of the text seems to be related to the main topics but is not included in the main topics list(main_topics), include it as a subtopic under the appropriate main topic.
    Do not include any additional notes or explanations in the output.

    Text: {text}
    """

    response = llm_4.invoke(prompt)
    extracted_details = response.content.strip()
    return extracted_details

In [270]:
detailed_info = extract_detrails(text, main_topics)
print(detailed_info)

<main>주제</main>
<sub>시계열 예측과 트레이딩 수익 극대화 강화학습을 활용한 시스템</sub> 
<content>기존 금융 딥러닝의 실패 요인들로는 노이즈가 심하고, 데이터 수가 부족하며, 과적합이 심한 점이 있다. 예를 들어, 마이크로소프트의 상장일인 1986년 3월 13일~현재까지의 주식 일봉 데이터 개수는 9491개 뿐이다. 이를 해결하기 위해 Time Series Forecasting을 통해 다음 일주일 동안 가장 큰 종가 변화율을 보일 종목을 예측하고, Reinforcement Learning을 통해 분 단위 거래를 학습하여 단타매매를 실행한다.</content> 
<page>3</page>

<main>WorkFlow</main>
<sub>Day Candle Stoch Datd</sub> 
<content>LETF, 6 Stock Code, 크레온 AP, Airute Candle EtoC</content> 
<page>4</page>

<main>종목 선정</main>
<sub>국내 주식 Reader로 데이터 수집</sub> 
<content>IFinance Data를 통해 2017-01-01부터 현재까지의 데이터를 수집하고, 수집 항목으로는 종가와 종목 KOSP가 포함된다.</content> 
<page>5</page>

<main>데이터수집및 전처리</main>
<sub>Transform</sub> 
<content>데이터 변환 과정이 포함된다.</content> 
<page>6</page>
<sub>계산 RSI Sharpe Ratio</sub> 
<content>RSI (Relative Index Strength)와 Sharpe Ratio를 계산한다. RSI는 상대강도지수로, Sharpe Ratio는 투자 포트폴리오의 기대 수익률과 무위험 수익률의 차이를 표준편차로 나눠 투자의 총 위험을 나타낸다.</content> 
<page>7</page>

<main>시계열 모델</main>
<sub>Long Short-Term Memory

### STEP4: 요약

In [271]:
def summarize_content(content):
    if not content.strip():
        return content

    prompt = f"""
    Do not change the language of the original input text. 
    Summarize in the same language as the input text.
    The purpose of the summaries is to create a comprehensive README for a GitHub project.
    If multiple subtopics within the same main topic have the same name, combine their content into a single entry with all relevant details, and merge the page numbers into a single list. 
    Ensure that no duplicate subtopics appear within the same main topic. 
    All content related to the same subtopic should be combined into one entry, regardless of how many times it appears.
    Ensure that subtopics from different main topics are not merged.

    Ensure that each summary accurately captures the key points and essential information of its respective partition.
    Summarize in a way that is concise and informative, omitting repetitive mentions of the main topic after the first mention in each partition.
    Focus on the key details and results relevant to the project, with particular emphasis on the final model outcomes. 
    Ensure that the final model results are described with a high level of accuracy and detail, including specific metrics, performance evaluations, and any significant findings.
    For each subtopic, aim to provide a summary that is 1 to 2 sentences long. 
    Focus on delivering a concise yet comprehensive overview, capturing the main points and essential details without unnecessary elaboration.
    When summarizing, only include the **first page** where each subtopic begins in the "pages used" list. For example, if a subtopic spans from page 7 to page 10, only include page 7 in the summary.
    Use the example format as a reference only—do not include it in the final output. Ensure that <sub> and <content> tags are separated by a single space, not a line break, and that the output follows this format exactly.
    Ensure the summary always begins with <subject>, <team>, and <index> in that order, before listing the main topics and content. 

    Here is an example of how the tagged text should look:

    <subject>Korea</subject>
    <team>이하나, 김하니</team>
    <index>Seasons in Korea>
    
    <main>Seasons in Korea</main>
    <sub>봄</sub> <content>한국의 봄은 3월에서 5월까지 지속되며, 온화한 기온과 함께 벚꽃이 만개하는 시기입니다. 이 시기에는 다양한 봄꽃 축제가 열리며, 사람들이 야외 활동을 즐기기에 좋은 날씨입니다.</content> <page>1</page>
    <sub>여름</sub> <content>한국의 여름은 대개 매우 덥고 습하며, 기온이 종종 30°C를 넘습니다. 이 시기는 장마철이기도 해서 특히 7월에 많은 비가 내립니다. 더위에도 불구하고, 이 시기는 휴가철로 인기가 많아 많은 사람들이 해변과 리조트로 향합니다.</content> <page>3</page>
    <sub>가을</sub> <content>가을은 한국에서 가장 아름다운 계절 중 하나로 꼽힙니다. 9월에서 11월 사이, 날씨는 시원하고 하늘은 맑으며, 단풍이 절정을 이루어 산과 들이 붉고 노란 색으로 물듭니다. 이 시기는 또한 수확의 계절로, 각종 축제가 열립니다.</content> <page>5</page>
    <sub>겨울</sub> <content>한국의 겨울은 12월에 시작되며, 대부분의 지역에서 기온이 영하로 떨어집니다. 겨울철은 건조하며, 특히 산악 지역에서는 가끔씩 눈이 내립니다. 이 시기에는 스키나 얼음 낚시와 같은 겨울 스포츠가 인기를 끌고 있습니다.</content> <page>7</page>

    <main>Animals in Korea</main>
    <sub>봄</sub> <content>봄철에는 한국의 자연이 다시 깨어나면서 다양한 동물들이 활동을 시작합니다. 산과 숲에서는 새들이 짝을 찾기 위해 지저귀고, 개구리와 같은 양서류들이 물가에서 활동을 재개합니다. 또한, 겨울잠에서 깨어난 동물들이 활발히 먹이를 찾는 모습을 볼 수 있습니다.</content> <page>9</page>
    <sub>여름</sub> <content>여름철에는 다양한 야생 동물들이 활발히 활동합니다. 특히, 한국의 산과 숲에서 멧돼지, 사슴, 다양한 종류의 새들이 많이 보입니다. 여름은 또한 번식기가 겹쳐 동물들이 더욱 활발히 움직이는 시기입니다.</content> <page>10</page>
    <sub>가을</sub> <content>가을에는 동물들이 겨울을 준비하는 모습을 볼 수 있습니다. 많은 동물들이 겨울잠을 준비하기 위해 지방을 축적하고, 새들은 따뜻한 지역으로 이동하기 시작합니다. 이 시기는 또한 수확기가 겹쳐, 농작물에 접근하는 동물들이 많아집니다.</content> <page>11</page>
    <sub>겨울</sub> <content>겨울철에는 많은 동물들이 추위를 피하기 위해 겨울잠에 들어갑니다. 겨울잠을 자지 않는 동물들은 추위를 이기기 위해 두꺼운 털을 기르거나 활동을 줄입니다. 특히, 한국의 산악지대에서는 고라니와 같은 동물들이 눈 속에서 식량을 찾아다니는 모습을 볼 수 있습니다.</content> <page>12</page>

    
    Text: {content}

    Please summarize the above content concisely.
    """

    response = llm_4.invoke(prompt)
    summary = response.content.strip()
    return summary


summarized_text = summarize_content(detailed_info)
print(summarized_text)

<subject>주제</subject>
<team>이하나, 김하니</team>
<index>시계열 예측과 트레이딩 수익 극대화 강화학습을 활용한 시스템>

<main>시계열 예측과 트레이딩 수익 극대화 강화학습을 활용한 시스템</main>
<sub>시계열 예측과 트레이딩 수익 극대화 강화학습을 활용한 시스템</sub> <content>기존 금융 딥러닝의 실패 요인으로는 노이즈, 데이터 부족, 과적합이 있으며, 이를 해결하기 위해 Time Series Forecasting과 Reinforcement Learning을 활용하여 단타매매를 실행한다.</content> <page>3</page>

<main>WorkFlow</main>
<sub>Day Candle Stoch Datd</sub> <content>LETF, 6 Stock Code, 크레온 AP, Airute Candle EtoC</content> <page>4</page>

<main>종목 선정</main>
<sub>국내 주식 Reader로 데이터 수집</sub> <content>IFinance Data를 통해 2017-01-01부터 현재까지의 종가와 종목 KOSP 데이터를 수집한다.</content> <page>5</page>

<main>데이터수집및 전처리</main>
<sub>Transform</sub> <content>데이터 변환 과정을 포함한다.</content> <page>6</page>
<sub>계산 RSI Sharpe Ratio</sub> <content>RSI와 Sharpe Ratio를 계산하여 투자 포트폴리오의 위험을 평가한다.</content> <page>7</page>

<main>시계열 모델</main>
<sub>Long Short-Term Memory (LSTM) 기반의 모델</sub> <content>RNN의 장기 의존성 문제를 해결하여 오랜 기간의 데이터를 반영할 수 있는 모델이다.</content> <page>8</page>
<sub>Dlinear LSTF-linear</s

In [272]:
# 텍스트 파일로 저장
# 경로 수정

output_name = file_path.split('\\')[-1].split('_text')[0]

# 새로운 파일 경로 설정
output_file_path = f'C:\\Users\\PC\\Desktop\\DoYoung\\DS\\github\\bitamin_auto_readme_generator\\data\\text_summarization\\output\\method1\\{output_name}_text.txt'

# summarized_text 변수를 텍스트 파일로 저장하는 코드
with open(output_file_path, 'w', encoding='utf-8') as file:
    file.write(summarized_text)

print(f"Summarized text has been saved to '{output_file_path}'.")


Summarized text has been saved to 'C:\Users\PC\Desktop\DoYoung\DS\github\bitamin_auto_readme_generator\data\text_summarization\output\method1\trading_text.txt'.
